In [ ]:
!pip install -U sagemaker

In [1]:
import sagemaker
from sagemaker.huggingface import HuggingFace

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
role = sagemaker.get_execution_role()

In [3]:
role

'arn:aws:iam::719742385994:role/SageMakerLLMRole'

In [4]:
hyperparameters = {
    "model_id": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
    "epochs": 2,
    "per_device_train_batch_size": 2,
    "lr": 2e-5
}

In [10]:
estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./script",
    role=role,
    transformers_version="4.36",
    pytorch_version="2.1",
    py_version="py310",
    instance_type="ml.g5.xlarge",
    instance_count=1,
    output_path="s3://llm-model-artifacts-girish/models/",
    hyperparameters=hyperparameters
)

In [11]:
estimator.fit({
    "train": "s3://llm-finetune-dataset-girish/datasets/"
})

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-11-19-11-16-01-374


2025-11-19 11:16:02 Starting - Starting the training job
2025-11-19 11:16:02 Pending - Training job waiting for capacity......
2025-11-19 11:16:37 Pending - Preparing the instances for training...
2025-11-19 11:17:27 Downloading - Downloading the training image........................
2025-11-19 11:21:35 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorith

In [13]:
# estimator.latest_training_job.model_data


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱ 1 estimator.latest_training_job.model_data                                                     │
│   2 # estimator.model_data                                                                       │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: '_TrainingJob' object has no attribute 'model_data'

In [14]:
estimator.model_data

's3://llm-model-artifacts-girish/models/huggingface-pytorch-training-2025-11-19-11-16-01-374/output/model.tar.gz'

In [9]:
# predictor = huggingface_estimator.deploy(
#     initial_instance_count=1,
#     instance_type="ml.g5.xlarge"
# )

# predictor.predict({"inputs": "Explain AWS S3"})


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱ 1 predictor = huggingface_estimator.deploy(                                                    │
│   2 │   initial_instance_count=1,                                                                │
│   3 │   instance_type="ml.g5.xlarge"                                                             │
│   4 )                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'huggingface_estimator' is not defined

In [ ]:
# Use this code to check all the accessible services inside your AWS Sagemaker

# from sagemaker import image_uris

# image_uris.retrieve(
#     framework="huggingface",
#     region="ap-south-1",   # change your region
#     version="4.37.0",
#     image_scope="inference"
# )

In [23]:
import sagemaker

sess = sagemaker.Session()
endpoint_name = "girish-live-finetune-endpoint"

# 1. Delete endpoint (if it exists)
try:
    sess.delete_endpoint(endpoint_name)
    print(f"Deleted endpoint: {endpoint_name}")
except Exception as e:
    print(f"Endpoint delete skipped/failed: {e}")

# 2. Delete endpoint config (if it exists)
try:
    sess.delete_endpoint_config(endpoint_name)
    print(f"Deleted endpoint config: {endpoint_name}")
except Exception as e:
    print(f"Endpoint config delete skipped/failed: {e}")


INFO:sagemaker:Deleting endpoint with name: girish-live-finetune-endpoint
INFO:sagemaker:Deleting endpoint configuration with name: girish-live-finetune-endpoint


Endpoint delete skipped/failed: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "girish-live-finetune-endpoint".
Deleted endpoint config: girish-live-finetune-endpoint


In [ ]:
# instance_type="ml.g5.xlarge"

In [24]:
## the code for the deployment
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

role = sagemaker.get_execution_role()

# model = HuggingFaceModel(
#     model_data="s3://llm-model-artifacts-girish/models/huggingface-pytorch-training-2025-11-19-11-16-01-374/output/model.tar.gz",
#     role=role,
#     transformers_version="4.37.0",
#     pytorch_version="2.1.0",
#     py_version="py310",
# )

model = HuggingFaceModel(
    model_data="s3://llm-model-artifacts-girish/models/huggingface-pytorch-training-2025-11-19-11-16-01-374/output/model.tar.gz",
    role=role,
    entry_point="inference.py",
    source_dir="inference",
    transformers_version="4.37.0",
    pytorch_version="2.1.0",
    py_version="py310"
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    endpoint_name="girish-live-finetune-endpoint"
)


INFO:sagemaker:Repacking model artifact (s3://llm-model-artifacts-girish/models/huggingface-pytorch-training-2025-11-19-11-16-01-374/output/model.tar.gz), script artifact (inference), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-719742385994/huggingface-pytorch-inference-2025-11-22-19-05-01-750/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2025-11-22-19-05-03-548
INFO:sagemaker:Creating endpoint-config with name girish-live-finetune-endpoint
INFO:sagemaker:Creating endpoint with name girish-live-finetune-endpoint


----------!

In [ ]:
## after the deployment URL will look like this
# https://runtime.sagemaker.<region>.amazonaws.com/endpoints/live-finetune-endpoint/invocations

In [ ]:
## after the deployment URL will look like this
https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/live-finetune-endpoint/invocations

In [31]:
# predictor.predict({"inputs": "Explain AWS S3"})
# Metformin 
# predictor.predict({"inputs": "Explain the mechanism of action of Metformin"})
print(predictor.predict({"inputs": "what Clinical trials have shown that adding Ezetimibe to statin therapy?"}))

what Clinical trials have shown that adding Ezetimibe to statin therapy?
Ezetimibe is a drug that is used to treat cholesterol. It is a statin. It is used to lower cholesterol in the blood.
Ezetimibe is a drug that is used to treat cholesterol. It is a statin. It is used to lower cholesterol in the blood.
Ezetimibe is a drug that is used to treat cholesterol. It is a statin. It is


In [30]:
import boto3, json

runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

resp = runtime.invoke_endpoint(
    EndpointName="girish-live-finetune-endpoint",
    ContentType="application/json",
    # Body=json.dumps({"inputs": "Explain the mechanism of action of Metformin"})
    Body=json.dumps({"inputs": "what Clinical trials have shown that adding Ezetimibe to statin therapy?"})
)

print(resp["Body"].read().decode())


"what Clinical trials have shown that adding Ezetimibe to statin therapy?\nEzetimibe is a drug that is used to treat cholesterol. It is a statin. It is used to lower cholesterol in the blood.\nEzetimibe is a drug that is used to treat cholesterol. It is a statin. It is used to lower cholesterol in the blood.\nEzetimibe is a drug that is used to treat cholesterol. It is a statin. It is"
